In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import files

# --- Data Loading and Cleaning ---
# Load the Brand_XYZ dataset
file_path = 'Brand_XYZ.csv'

# Check if the file exists, if not, prompt to upload
if not os.path.exists(file_path):
    print(f"File '{file_path}' not found. Please upload it.")
    uploaded = files.upload()
    # Ensure the uploaded file is the one we expect, or raise an error
    if file_path not in uploaded:
        raise FileNotFoundError(f"'{file_path}' was not uploaded. Please upload the correct file.")

df = pd.read_csv(file_path, encoding='latin1')

# Ensure numeric columns are properly typed (using coerce to handle potential non-numeric entries)
numeric_cols = ['Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# --- 1. Overall Category Performance Analysis ---

# Group by main Category and calculate aggregate metrics
category_performance = df.groupby('Category').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Total_Quantity=('Quantity', 'sum')
)

# Calculate Profit Margin (Profit / Sales)
category_performance['Profit_Margin'] = (category_performance['Total_Profit'] / category_performance['Total_Sales']) * 100
category_performance = category_performance.sort_values(by='Total_Sales', ascending=False)

# Format the output for readability
def format_currency(x):
    if x >= 1e6:
        return f'${x/1e6:,.2f}M'
    return f'${x:,.0f}'

def format_percent(x):
    return f'{x:.2f}%'

category_performance_formatted = category_performance.copy()
category_performance_formatted['Total Sales'] = category_performance_formatted['Total_Sales'].apply(format_currency)
category_performance_formatted['Total Profit'] = category_performance_formatted['Total_Profit'].apply(format_currency)
category_performance_formatted['Profit Margin'] = category_performance_formatted['Profit_Margin'].apply(format_percent)
category_performance_formatted = category_performance_formatted[['Total Sales', 'Total Profit', 'Total_Quantity', 'Profit Margin']]


# --- 2. Deep Dive into the Top Selling Category (e.g., Technology) ---

# Find the category with the highest sales
top_category_name = category_performance.index[0]

# Filter data for the top category
top_category_df = df[df['Category'] == top_category_name]

# Group by Sub-Category within the top category
subcategory_dive = top_category_df.groupby('Sub-Category').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Profit=('Profit', 'sum')
)

# Calculate Sub-Category Profit Margin
subcategory_dive['Profit_Margin'] = (subcategory_dive['Total_Profit'] / subcategory_dive['Total_Sales']) * 100
subcategory_dive = subcategory_dive.sort_values(by='Profit_Margin', ascending=False)

# Get the top 3 and bottom 3 sub-categories by Profit Margin
top_3_subcategories = subcategory_dive.head(3)
bottom_3_subcategories = subcategory_dive.tail(3)


# --- 3. Regional Performance Analysis ---

regional_performance = df.groupby('Region').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Profit=('Profit', 'sum')
).sort_values(by='Total_Sales', ascending=False)

regional_performance['Profit_Margin'] = (regional_performance['Total_Profit'] / regional_performance['Total_Sales']) * 100

regional_performance_formatted = regional_performance.copy()
regional_performance_formatted['Total Sales'] = regional_performance_formatted['Total_Sales'].apply(format_currency)
regional_performance_formatted['Total Profit'] = regional_performance_formatted['Total_Profit'].apply(format_currency)
regional_performance_formatted['Profit Margin'] = regional_performance_formatted['Profit_Margin'].apply(format_percent)
regional_performance_formatted = regional_performance_formatted[['Total Sales', 'Total Profit', 'Profit Margin']]


# --- Print Results ---
print(f"--- Brand XYZ Sales Analysis ---\n")

print("1. OVERALL CATEGORY PERFORMANCE (Sorted by Total Sales)")
print(category_performance_formatted.to_markdown(numalign="left", stralign="left"))
print("\n" + "="*80 + "\n")


print(f"2. DEEP DIVE: Sub-Category Profitability within the Top Category: {top_category_name}")
print("\nTOP 3 MOST PROFITABLE SUB-CATEGORIES:")
print(top_3_subcategories[['Total_Sales', 'Total_Profit', 'Profit_Margin']].applymap(lambda x: format_currency(x) if isinstance(x, (int, float)) and 'Margin' not in str(x) else format_percent(x)).to_markdown(numalign="left", stralign="left"))

print("\nBOTTOM 3 LEAST PROFITABLE SUB-CATEGORIES:")
print(bottom_3_subcategories[['Total_Sales', 'Total_Profit', 'Profit_Margin']].applymap(lambda x: format_currency(x) if isinstance(x, (int, float)) and 'Margin' not in str(x) else format_percent(x)).to_markdown(numalign="left", stralign="left"))
print("\n" + "="*80 + "\n")


print("3. REGIONAL PERFORMANCE (Sorted by Total Sales)")
print(regional_performance_formatted.to_markdown(numalign="left", stralign="left"))
print("\n--- END OF ANALYSIS ---")

# Save detailed results to a CSV for external use
category_performance.to_csv('category_performance_detail.csv')
subcategory_dive.to_csv('subcategory_dive_detail.csv')

print("\nDetailed results saved to 'category_performance_detail.csv' and 'subcategory_dive_detail.csv'.")

File 'Brand_XYZ.csv' not found. Please upload it.
